In [12]:
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
#importing the json file
data_file=open('intents.json').read()
data=json.loads(data_file)

#the dataset has objects known as intents
#intents are having their own tags showing context,patterns and responese
#these intents will help us to reply to those statements.

In [14]:
import pickle as pk
from sklearn.feature_extraction.text import CountVectorizer

# Example vectorizer (you'll replace this with your actual vectorizer)
vectorizer = CountVectorizer()

# Save the vectorizer
with open('vectorizer.pkl', 'wb') as f:
    pk.dump(vectorizer, f)


In [5]:
#Creating data_X and data_Y
#using NLTK's WordNetLemmatizer(). This function will help
#us to reduce the words and convert it into their root words

words=[] #For Row model/ vocabulary for patterns
classes=[] #For Row model/vocalbulary for tags
data_X=[] #For storing each pattern
data_Y=[] #For storing tag corresponding to each pattern in data_X

#Iterating all intents in intents.json

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens=nltk.word_tokenize(pattern) #tokenizing each pattern
    words.extend(tokens) #append tokens into words
    data_X.append(pattern) #appending pattern to data_X
    data_Y.append(intent["tag"]) #appending the associated tag to each pattern

  if intent["tag"] not in classes: #appending the tag if it is not present
    classes.append(intent["tag"])

lemmatizer=WordNetLemmatizer()
#lemmatizer to get the root of words
#turning everything into lowercase and removing all punctuations

words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
#Sorting the vocabulary and classes into aplhabetical order
#checking if there are no duplicate orders

words=sorted(set(words))
classes=sorted(set(classes))


In [15]:
with open('vocab.pkl', 'wb') as f:
    pk.dump(words, f)

with open('classes.pkl', 'wb') as f:
    pk.dump(classes, f)

In [6]:
#Data TRAINING

#We are converting our text into numbers using BOW model(Bag of words)
#words and classes act as a vocabulary for patterns and tags respectively

#1 if the word is present in the pattern/tag being read(from data_X)
#0 if absent.

#Data will be converted into numbers
#stored into two arrays: train_X and train_Y respectively.


training=[]
out_empty=[0]*len(classes) #intializing

#BAG of model
for idx,doc in enumerate(data_X):
  bow=[]
  text=lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)


  output_row=list(out_empty)
  output_row[classes.index(data_Y[idx])]=1

  #add the one hot encoded BOW and associated classes to traininhg

  training.append([bow, output_row])

#shuffle the data and convert it to an array

random.shuffle(training)
training=np.array(training,dtype=object)

#spliting the features and target_labels
train_X=np.array(list(training[:,0]))
train_Y=np.array(list(training[:,1]))

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout

# Define your model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation='relu'))  # Adjusted to match your description
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation='softmax'))

# Define the optimizer
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

# Print model summary
print(model.summary())

# Train the model
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

# Save the model
save_model(model, 'my_model.keras')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 128)                 │          10,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 19)                  │           1,235 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,243 (79.07 KB)

 Trainable params: 20,243 (79.07 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.0361 - loss: 2.9771
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1290 - loss: 2.8125 
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2244 - loss: 2.7008 
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2965 - loss: 2.5187 
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2965 - loss: 2.3617 
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3221 - loss: 2.1695 
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4279 - loss: 2.0049 
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5593 - loss: 1.8848 
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5385 - loss: 1.6544
Epoch 10/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6186 - loss: 1.3598 
Epoch 11/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6546 - loss: 1.1851 
Epoch 12/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7476 -

In [ ]:
#now i will start to create a chatbot
#We will create few easy functions that will convert the user's input query to arrays and predict the relevant tag for it.


#Preprocessing of the input
'''

def clean_text(text):
  tokens=nltk.word_tokenize(text)
  tokens=(lemmatizer.lemmatize(word) for word in tokens)
  return tokens

def bag_of_words(text,vocab):
  tokens=clean_text(text)
  bow=[0]*len(vocab)
  for w in tokens:
    for idx,word in enumerate(vocab):
      if word==w:
        bow[idx]=1
  return np.array(bow)

def pred_class(text,vocab,labels):
  bow=bag_of_words(text, vocab)
  result=model.predict(np.array([bow]))[0] #extracting proabilities

  thresh=0.5
  y_pred=[[indx, res] for indx, res in enumerate(result) if res>thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  #Sorting by values of probability in decresing order
  return_list=[]

  for r in y_pred:
    return_list.append(labels[r[0]])
    #contains labels(tags) for highest proability
  return return_list

def get_response(intents_list, intents_json):
  if len(intents_list)==0:
    result="Sorry! , I didn't understand."
  else:
    tag=intents_list[0]
    list_of_intents=intents_json["intents"]
    for i in list_of_intents:
      if i["tag"]==tag:
        result=random.choice(i["responses"])
        break
  return result
  '''


In [ ]:
'''print("Press 0 if you don't want to chat with our ChatBot.")
while True:
  message=input("")
  if message =="0":
    break
  intents=pred_class(message,words,classes)
  result=get_response(intents,data)
  print(result)
  '''

Press 0 if you don't want to chat with our ChatBot.
1/1 [==============================] - 0s 20ms/step
Hey!
1/1 [==============================] - 0s 19ms/step
Sorry! , I didn't understand.
1/1 [==============================] - 0s 20ms/step
My job is to make you feel like everything is okay.
1/1 [==============================] - 0s 21ms/step
I was born in 2021
